# Import + Preprocessing

In [1]:
from fonctions import Dataset
from src.preprocessing import display_missing_values

In [2]:
train=Dataset("data/train.csv")
data_train=train.load_data()
test=Dataset("data/test.csv")
data_test=test.load_data()

In [3]:
from fonctions import Preprocessor,TrainPreprocessor,TestPreprocessor
train_preprocessor=TrainPreprocessor(data_train)
test_preprocessor=TestPreprocessor(data_test)

In [4]:
train_preprocessor.fill_fuel_consumption()
train_preprocessor.fill_electric_range()
train_preprocessor.fill_engine_capacity()
train_preprocessor.fill_electric_consumption()
train_preprocessor.fill_category_type()
train_preprocessor.fill_wheel_base()
train_preprocessor.fill_At_1()
train_preprocessor.fill_At_2()
train_preprocessor.fill_mass()
train_preprocessor.fill_engine_power()
#train_preprocessor.last_step()

test_preprocessor.fill_fuel_consumption()
test_preprocessor.fill_electric_range()
test_preprocessor.fill_engine_capacity()
test_preprocessor.fill_electric_consumption()
test_preprocessor.fill_category_type()
test_preprocessor.fill_wheel_base()
test_preprocessor.fill_At_1()
test_preprocessor.fill_At_2()
test_preprocessor.fill_mass()
test_preprocessor.fill_engine_power()
#test_preprocessor.last_step()

In [5]:
data_train = train_preprocessor.encode_that_var("Ct")
data_train = train_preprocessor.encode_that_var("Cr")
data_test = test_preprocessor.encode_that_var("Ct")
data_test = test_preprocessor.encode_that_var("Cr")

In [5]:
def create_conforme(df):
    df['conforme'] = df['Tan'].isna()
    df['conforme'] = df['conforme'].apply(lambda x: 1 if x==False else 0)
    df.drop(columns='Tan', inplace=True)
    pass
def compute_surface(obs):
    max_largeur= max(obs['At1 (mm)'], obs['At2 (mm)'])
    return obs['W (mm)']*obs['At1 (mm)'] if max_largeur == obs['At1 (mm)'] else obs['W (mm)'] * obs['At2 (mm)']

def create_surface(df):
    df['surface']= df.apply(compute_surface, axis=1)
    pass

def group_fuel_types(category: str):
    if category in ['PETROL/ELECTRIC', 'DIESEL/ELECTRIC']:
        return "HYBRID"
    elif category in ['NG-BIOMETHANE', 'HYDROGEN', 'NG','E85']:
        return "BIO-FUEL"
    elif category in ['PETROL','LPG'] :
        return 'PETROL'
    else:
        return category
def create_carburant(df):
    df['carburant']= df['Ft'].apply(group_fuel_types)
    df.drop(columns='Ft',inplace = True)

In [6]:
create_conforme(data_train)
create_surface(data_train)
create_carburant(data_train)

create_conforme(data_test)
create_surface(data_test)
create_carburant(data_test)

In [8]:
data_train = train_preprocessor.encode_that_var("carburant")
data_test = test_preprocessor.encode_that_var("carburant")

In [7]:
drop_this_cuz_no_use=['VFN', 'Mp', 'Mh', 'Man', 'T', 'Mk', 'Cn','Mt']
drop_this_cuz_personnal_choice = ['W (mm)', 'At1 (mm)', 'At2 (mm)','Fm','Erwltp (g/km)']

data_train.drop(columns=drop_this_cuz_no_use, inplace=True)
data_test.drop(columns=drop_this_cuz_no_use, inplace=True)

In [8]:
data_train.drop(columns='ID',inplace=True)

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
from src.preprocessing import display_missing_values
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_theme(style="ticks", palette="pastel")

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# Random Forest (numerical)
No drop dupplicates, only keep numerical cols

MAE: 3.2843

In [12]:

columns_to_keep = ['m (kg)', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'Fuel consumption ', 'Electric range (km)']

train, test = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [13]:
random_forest = RandomForestRegressor(random_state=42)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 3.2811834066371595


In [16]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test[data_test.columns.intersection(columns_to_keep)])

In [17]:
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_numerical_no_drop_dups.csv", index=False)

## RF numerical (new set of predictors)

In [13]:

columns_to_keep = ['m (kg)', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'Fuel consumption ']

train, test = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [14]:
random_forest = RandomForestRegressor(random_state=42, n_jobs=-1)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 3.314147394462553


In [17]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test[data_test.columns.intersection(columns_to_keep)])

In [18]:
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_num_dups_no_e_range.csv", index=False)

# RF (numerical) + K fold

In [19]:
columns_to_keep_no_target = ['m (kg)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'Fuel consumption ']

X_train = data_train[data_train.columns.intersection(columns_to_keep_no_target)]
y_train= data_train["Ewltp (g/km)"]

In [20]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(criterion='absolute_error', random_state=42, n_jobs=-1)

k_folds = 4

kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Créez des listes pour stocker les modèles et les scores
models = []
scores = []

# Effectuez la validation croisée
for train_index, test_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Entraînez le modèle sur X_train_fold et y_train_fold
    model.fit(X_train_fold, y_train_fold)

    # Évaluez le modèle sur X_val_fold (vous pouvez utiliser la métrique de votre choix, par exemple MAE)
    y_pred = model.predict(X_val_fold)
    mae = mean_absolute_error(y_val_fold, y_pred)
    
    # Stockez le modèle et le score
    models.append(model)
    scores.append(mae)

# Affichez les scores MAE pour chaque pli
for i, mae in enumerate(scores):
    print(f"Fold {i + 1} MAE: {mae}")

# Calculez la moyenne des scores MAE
average_mae = sum(scores) / k_folds
print(f"Average MAE: {average_mae}")

best_model_index = scores.index(min(scores))
best_model = models[best_model_index]

print(f"min MAE: {min(scores)}")

KeyboardInterrupt: 

In [ ]:
data_test["Ewltp (g/km)"] = best_model.predict(data_test.drop(columns='ID'))

In [ ]:
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_numerical_kf.csv", index=False)